In [3]:
import pandas as pd
from datasets import Dataset

# Load the augmented CSV
df = pd.read_csv('augmented.csv')

# Convert DataFrame to Huggingface Dataset format
dataset = Dataset.from_pandas(df)
dataset = dataset.rename_column('Reason', 'text')  # Rename 'Reason' column to 'text' for fine-tuning

In [4]:
# !huggingface-cli login      

In [1]:
# from transformers import LlamaForCausalLM, LlamaTokenizer
# #8B,70B,405B
# # Load the LLaMA tokenizer
# tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-3.1-405B")

# # Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.1-405B", 
#                                          load_in_4bit=True, 
#                                          device_map="auto")


from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'triton'

In [ ]:
# from transformers import LlamaForCausalLM
# from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# # Load the LLaMA-3 model for causal language modeling
# model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3b", load_in_4bit=True, device_map="auto")

# # Prepare the model for QLoRA fine-tuning
# model = prepare_model_for_kbit_training(model)

# # Define the LoraConfig for QLoRA
# lora_config = LoraConfig(
#     r=8,  # Rank of the low-rank matrix
#     lora_alpha=32,  # Alpha scaling factor
#     target_modules=["q_proj", "v_proj"],  # Target attention layers
#     lora_dropout=0.05,  # Dropout for LoRA layers
#     bias="none",  # Specify bias handling
#     task_type="CAUSAL_LM"  # Task type
# )

# # Wrap the model with QLoRA
# model = get_peft_model(model, lora_config)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)